# I learn BeautifulSoup for Web Crawling with example

In [1]:
from bs4 import BeautifulSoup
import peforth, re
import numpy as np
from pandas.io import clipboard  # clipboard.copy('test) puts 'test' into clipboard. Thhis is in case the HTML text might be copied from F12 DevTools.

reDef unknown
reDef -->


In [5]:
soup = BeautifulSoup(source,"html.parser")  # source string 在下面
# soup = BeautifulSoup(open(r"source.html"),"html.parser")

In [6]:
%f soup :> prettify()[:500] ==>

soup :> prettify()[:500] ==>
 <p name="lalala">
 I am lalala
</p>
<div class="left-bloc console-bloc bloc" ng-class="{'with-leaderboard': !!apis.miniLeaderboard}" style="right: calc(100% - (510px + 0.5 * (100% - 1020px)));">
 <div class="bloc-header">
  <h3 cg-ide-console-title="" class="console-left-header bloc-title" translate="cgIde.console">
   Console output
  </h3>
  <div class="console-filter-selector">
   <div class="cg-ide-console-filter" filter="consoleFilter" ng-class="{open: isOpen}" ng-show="apis.console">
    < (<class 'str'>)


In [7]:
# soup.<tag> 的用法
%f ( 沒有 title ) soup :> title -->  
%f ( 沒有 anchor ) soup :> a --> 
%f ( 這就是 Console Output 了 ) soup :> h3 -->

( 沒有 title ) soup :> title --> None (<class 'NoneType'>)
( 沒有 anchor ) soup :> a --> None (<class 'NoneType'>)
( 這就是 Console Output 了 ) soup :> h3 --> <h3 cg-ide-console-title="" class="console-left-header bloc-title" translate="cgIde.console">Console output</h3> (<class 'bs4.element.Tag'>)


In [9]:
# soup.<tag>.name 的用法
%f ( 抓第一個 p tag ) soup :> p -->  
%f ( 這 name 指的是這個 tag 的 name ) soup :> p.parent.name -->  
%f ( attributes 是個 dict ) soup :> p.attrs -->  
%f ( innerText of the tag ) soup :> p.string -->  


( 抓第一個 p tag ) soup :> p --> <p name="lalala"> I am lalala</p> (<class 'bs4.element.Tag'>)
( 這 name 指的是這個 tag 的 name ) soup :> p.parent.name --> [document] (<class 'str'>)
( attributes 是個 dict ) soup :> p.attrs --> {'name': 'lalala'} (<class 'dict'>)
( innerText of the tag ) soup :> p.string -->  I am lalala (<class 'bs4.element.NavigableString'>)


In [31]:
# 下行遍歷  tag.contents, tag.children, tag.desendants 三個都一樣是個 array 但呈現為 iterator, list, and generator. 其中 .contents 是直接 readable 的。
%f soup :> div.div.div.div.div dup :> prettify()[:100] ==>
%f ( .children 是沒有實體的 iterator ) cr dup :> children ==>
%f cr dup :> children list ==>
%f ( .contents 其實就是 .children 的 list 實體 ) cr dup :> contents ==>
%f ( .descendants 又是個沒有實體的 generator ) cr dup :> decendants ==>
%f ( .descendants generator 列出來看 ) cr list --> 

soup :> div.div.div.div.div dup :> prettify()[:100] ==>
 <div class="selected-items" ng-click="open()">
 <!-- -->
 <!-- -->
 <span class="selected-item" ng-i (<class 'str'>)

( .children 是沒有實體的 iterator ) cr dup :> children ==>
 <list_iterator object at 0x000002569DFD8348> (<class 'list_iterator'>)

cr dup :> children list ==>
 [' ', ' ', ' ', <span class="selected-item" ng-if="selected" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.gameInfo">Game information</span>, ' ', ' ', ' ', <span class="selected-item" ng-if="selected" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.stdout">Action (Output Stream)</span>, ' ', ' ', ' ', <span class="selected-item" ng-if="selected" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.stderr">Debug (Errors Stream)</span>, ' ', ' ', ' '] (<class 'list'>)

( .contents 其實就是 .children 的 list 實體 ) cr dup :> contents ==>
 [' ', ' ', ' ', <span class="selected-item" ng-if="selected" ng-repeat="(name, s

In [34]:
# 上行遍歷  tag.parent, tag.parents (iterator)
%f ( Get a tag ) soup :> div.div.div.div.div ( -- tag )
%f dup :> parent ==> cr
%f dup :> parents ==> cr
%f ( 上頭有 5 層 parents 核算無誤 ) dup :> parents list count nip --> cr
    

dup :> parent ==>
 <div class="cg-ide-console-filter" filter="consoleFilter" ng-class="{open: isOpen}" ng-show="apis.console"> <div class="selected-items" ng-click="open()"> <!-- --><!-- --><span class="selected-item" ng-if="selected" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.gameInfo">Game information</span><!-- --><!-- --><!-- --><span class="selected-item" ng-if="selected" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.stdout">Action (Output Stream)</span><!-- --><!-- --><!-- --><span class="selected-item" ng-if="selected" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.stderr">Debug (Errors Stream)</span><!-- --><!-- --> </div> <!-- --> </div> (<class 'bs4.element.Tag'>)

dup :> parents ==>
 <generator object PageElement.parents at 0x000002569DFCB848> (<class 'generator'>)

( 上頭有 5 層 parents 核算無誤 ) dup :> parents list count nip --> 5 (<class 'int'>)



In [35]:
# 平行遍歷 .next_sibling .previous_sibling .next_siblings .previous_siblings
%f ( Get a tag ) soup :> div.div.div.div.div ( -- tag )
%f dup :> next_siblings list -->
%f dup :> previous_siblings list -->


dup :> next_siblings list --> [' ', ' ', ' '] (<class 'list'>)
dup :> previous_siblings list --> [' '] (<class 'list'>)


# Playground

In [22]:
%f soup :> find_all('span') \ ( -- list ) 
%f <py> [ i.string for i in pop()][:10]</pyV> --> cr
%f \ soup :> find_all('pre') \ ( -- list ) 
%f \ <py> [ i.string for i in pop()]</pyV> --> cr 
%f \ soup :> find('pre') ==>
%f ( 從這裏面去爬 ) soup :> find('pre').string ==>


<py> [ i.string for i in pop()][:10]</pyV> --> ['Game information', 'Action (Output Stream)', 'Debug (Errors Stream)', '00', '69', '01', '69', '02', '69', '03'] (<class 'list'>)

( 從這裏面去爬 ) soup :> find('pre').string ==>
 Landing phase starting
X=2500m, Y=2500m, HSpeed=0m/s VSpeed=0m/s
Fuel=500l, Angle=0°, Power=0 (0.0m/s2)
 (<class 'bs4.element.NavigableString'>)


In [15]:
%f cr
%f soup :> find(class_='frame-players')  ( -- list )  <py> [ i for i in pop()]</pyV> ==> cr 
%f soup :> find(class_='frame-players')  ( -- list )  <py> [ i.string for i in pop()]</pyV> ==> cr 
%f soup :> find_all(class_='frame-number') count --> ( -- list )  <py> [ i.string for i in pop()]</pyV> ==> cr 
%f soup :> find_all(class_='frame-count')  count --> ( -- list )  <py> [ i.string for i in pop()]</pyV> ==> cr 
# %f soup :> find(class_='frame-players').find(class_='outputLine') type -->

soup type --> <class 'bs4.BeautifulSoup'> (<class 'type'>)

soup :> find(class_='frame-players').find(class_='game-info') --> <div class="game-info" ng-if="::subframe.gameInformation"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing phase starting
X=2500m, Y=2500m, HSpeed=0m/s VSpeed=0m/s
Fuel=500l, Angle=0簞, Power=0 (0.0m/s2)
</pre> </div> (<class 'bs4.element.Tag'>)

soup :> find(class_='frame-players').find(class_='game-info').find(class_='title').string --> Game information: (<class 'bs4.element.NavigableString'>)
( Phase log data ) <py> re.sub('\r|\n',' ',pop())</pyV> --> Landing phase starting X=2500m, Y=2500m, HSpeed=0m/s VSpeed=0m/s Fuel=500l, Angle=0簞, Power=0 (0.0m/s2)  (<class 'str'>)
soup :> find(class_='frame-players').find(class_='frame-number').string --> 00 (<class 'bs4.element.NavigableString'>)
soup :> find(class

# The source HTML string copied from F12 DevTools

In [4]:
source = '''
<p name=lalala> I am lalala</p>
<div class="left-bloc console-bloc bloc" ng-class="{'with-leaderboard': !!apis.miniLeaderboard}" style="right: calc(100% - (510px + 0.5 * (100% - 1020px)));"> <div class="bloc-header"> <h3 class="console-left-header bloc-title" cg-ide-console-title="" translate="cgIde.console">Console output</h3> <div class="console-filter-selector"> <div class="cg-ide-console-filter" ng-class="{open: isOpen}" ng-show="apis.console" filter="consoleFilter"> <div class="selected-items" ng-click="open()"> <!----><!----><span ng-if="selected" class="selected-item" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.gameInfo">Game information</span><!----><!----><!----><span ng-if="selected" class="selected-item" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.stdout">Action (Output Stream)</span><!----><!----><!----><span ng-if="selected" class="selected-item" ng-repeat="(name, selected) in filter" translate="ideConsoleFilter.stderr">Debug (Errors Stream)</span><!----><!----> </div> <!----> </div> </div> <div class="header-buttons"> <!----><button ng-disabled="!layoutEnabled" translate="" translate-attr-title="cgIde.minimize" ng-if="consoleLayout!=='minimized'" class="header-button minimize-button" ng-click="setConsoleLayout('minimized')" title="Minimize"> </button><!----><!----><!----><button ng-disabled="!layoutEnabled" translate="" translate-attr-title="cgIde.restore" ng-if="consoleLayout==='expanded'" class="header-button restore-button" ng-click="setConsoleLayout('normal')" title="Restore"> </button><!----><!----> </div> </div> <!----> <!----> <div class="console-right-bloc console-content"> <div class="bloc-content-wrapper"> <!----><div class="cg-ide-console" ng-if="apis.console" api="apis.console" data-filter="consoleFilter"> <!----> <div class="cg-ide-console-content"> <div class="server-side-problem ng-hide" ng-show="refereeError"> <!----><p ng-if="!failCause" translate="ideConsole.refereeError">Internal error. An error report has been sent to the CodinGame team.</p><!----> <!----> </div> <div class="frames with-frame-counter" ng-class="{'with-frame-counter': validFrameCount > 2}"> <div class="cg-ide-console-frame-container"> <!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing phase starting
X=2500m, Y=2500m, HSpeed=0m/s VSpeed=0m/s
Fuel=500l, Angle=0°, Power=0 (0.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">00</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">[[0, 100], [1000, 500], [1500, 100], [3000, 100], [5000, 1500], [6999, 1000]]</div><div class="outputLine">command 0 0</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 0</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2498m, HSpeed=0m/s VSpeed=-4m/s
Fuel=500l, Angle=0°, Power=0 (0.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">01</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 1</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 1</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2493m, HSpeed=0m/s VSpeed=-6m/s
Fuel=499l, Angle=0°, Power=1 (1.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">02</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 0</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 0</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2485m, HSpeed=0m/s VSpeed=-10m/s
Fuel=499l, Angle=0°, Power=0 (0.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">03</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 0</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 0</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2473m, HSpeed=0m/s VSpeed=-14m/s
Fuel=499l, Angle=0°, Power=0 (0.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">04</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 1</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 1</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2458m, HSpeed=0m/s VSpeed=-17m/s
Fuel=498l, Angle=0°, Power=1 (1.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">05</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 0</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 0</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2439m, HSpeed=0m/s VSpeed=-20m/s
Fuel=498l, Angle=0°, Power=0 (0.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">06</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 1</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 1</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2418m, HSpeed=0m/s VSpeed=-23m/s
Fuel=497l, Angle=0°, Power=1 (1.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">07</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 0</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 0</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2393m, HSpeed=0m/s VSpeed=-27m/s
Fuel=497l, Angle=0°, Power=0 (0.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">08</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 1</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 1</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2365m, HSpeed=0m/s VSpeed=-29m/s
Fuel=496l, Angle=0°, Power=1 (1.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">09</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 2</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 2</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2334m, HSpeed=0m/s VSpeed=-31m/s
Fuel=494l, Angle=0°, Power=2 (2.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">10</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2303m, HSpeed=0m/s VSpeed=-32m/s
Fuel=491l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">11</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2271m, HSpeed=0m/s VSpeed=-32m/s
Fuel=487l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">12</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2239m, HSpeed=0m/s VSpeed=-32m/s
Fuel=484l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">13</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2207m, HSpeed=0m/s VSpeed=-32m/s
Fuel=480l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">14</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2175m, HSpeed=0m/s VSpeed=-33m/s
Fuel=477l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">15</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 2</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 2</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2141m, HSpeed=0m/s VSpeed=-34m/s
Fuel=475l, Angle=0°, Power=2 (2.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">16</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2107m, HSpeed=0m/s VSpeed=-35m/s
Fuel=472l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">17</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 2</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 2</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2071m, HSpeed=0m/s VSpeed=-37m/s
Fuel=470l, Angle=0°, Power=2 (2.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">18</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=2034m, HSpeed=0m/s VSpeed=-38m/s
Fuel=467l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">19</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1996m, HSpeed=0m/s VSpeed=-37m/s
Fuel=463l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">20</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1959m, HSpeed=0m/s VSpeed=-38m/s
Fuel=460l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">21</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1921m, HSpeed=0m/s VSpeed=-38m/s
Fuel=456l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">22</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1883m, HSpeed=0m/s VSpeed=-37m/s
Fuel=452l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">23</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1846m, HSpeed=0m/s VSpeed=-38m/s
Fuel=449l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">24</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1808m, HSpeed=0m/s VSpeed=-38m/s
Fuel=445l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">25</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1770m, HSpeed=0m/s VSpeed=-37m/s
Fuel=441l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">26</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1732m, HSpeed=0m/s VSpeed=-38m/s
Fuel=438l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">27</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1694m, HSpeed=0m/s VSpeed=-38m/s
Fuel=434l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">28</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1656m, HSpeed=0m/s VSpeed=-39m/s
Fuel=431l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">29</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1618m, HSpeed=0m/s VSpeed=-38m/s
Fuel=427l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">30</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1579m, HSpeed=0m/s VSpeed=-38m/s
Fuel=423l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">31</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1541m, HSpeed=0m/s VSpeed=-39m/s
Fuel=420l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">32</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1502m, HSpeed=0m/s VSpeed=-38m/s
Fuel=416l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">33</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1464m, HSpeed=0m/s VSpeed=-39m/s
Fuel=413l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">34</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1425m, HSpeed=0m/s VSpeed=-39m/s
Fuel=409l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">35</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1386m, HSpeed=0m/s VSpeed=-39m/s
Fuel=405l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">36</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1347m, HSpeed=0m/s VSpeed=-39m/s
Fuel=402l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">37</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1308m, HSpeed=0m/s VSpeed=-39m/s
Fuel=398l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">38</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1269m, HSpeed=0m/s VSpeed=-39m/s
Fuel=394l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">39</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1230m, HSpeed=0m/s VSpeed=-39m/s
Fuel=391l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">40</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1190m, HSpeed=0m/s VSpeed=-39m/s
Fuel=387l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">41</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1151m, HSpeed=0m/s VSpeed=-39m/s
Fuel=383l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">42</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1113m, HSpeed=0m/s VSpeed=-39m/s
Fuel=379l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">43</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1074m, HSpeed=0m/s VSpeed=-38m/s
Fuel=375l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">44</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=1036m, HSpeed=0m/s VSpeed=-39m/s
Fuel=372l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">45</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=997m, HSpeed=0m/s VSpeed=-39m/s
Fuel=368l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">46</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=958m, HSpeed=0m/s VSpeed=-39m/s
Fuel=365l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">47</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=918m, HSpeed=0m/s VSpeed=-39m/s
Fuel=361l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">48</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=879m, HSpeed=0m/s VSpeed=-40m/s
Fuel=358l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">49</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=839m, HSpeed=0m/s VSpeed=-40m/s
Fuel=354l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">50</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=800m, HSpeed=0m/s VSpeed=-39m/s
Fuel=350l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">51</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=761m, HSpeed=0m/s VSpeed=-39m/s
Fuel=346l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">52</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=722m, HSpeed=0m/s VSpeed=-39m/s
Fuel=342l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">53</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=683m, HSpeed=0m/s VSpeed=-38m/s
Fuel=338l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">54</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=645m, HSpeed=0m/s VSpeed=-38m/s
Fuel=334l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">55</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=607m, HSpeed=0m/s VSpeed=-39m/s
Fuel=331l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">56</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=568m, HSpeed=0m/s VSpeed=-39m/s
Fuel=327l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">57</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=529m, HSpeed=0m/s VSpeed=-39m/s
Fuel=324l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">58</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=490m, HSpeed=0m/s VSpeed=-39m/s
Fuel=320l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">59</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=451m, HSpeed=0m/s VSpeed=-39m/s
Fuel=316l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">60</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=413m, HSpeed=0m/s VSpeed=-38m/s
Fuel=312l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">61</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=374m, HSpeed=0m/s VSpeed=-39m/s
Fuel=309l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">62</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=335m, HSpeed=0m/s VSpeed=-39m/s
Fuel=305l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">63</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=296m, HSpeed=0m/s VSpeed=-40m/s
Fuel=302l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">64</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=257m, HSpeed=0m/s VSpeed=-39m/s
Fuel=298l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">65</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=217m, HSpeed=0m/s VSpeed=-39m/s
Fuel=294l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">66</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 3</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 3</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=178m, HSpeed=0m/s VSpeed=-40m/s
Fuel=291l, Angle=0°, Power=3 (3.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">67</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml">Landing in progress...
X=2500m, Y=139m, HSpeed=0m/s VSpeed=-39m/s
Fuel=287l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">68</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----><div class="frame keyframe currentFrame" ng-class="::{
					keyframe: frame.keyframe,
					empty: !isFrameValid(frame)
				}" ng-repeat="frame in frames" style=""> <!----><!----><div class="frame-players mySubFrame" ng-repeat="subframe in frame.subframes" ng-if="isSubframeValid(subframe)" style=""> <!----><div ng-if="::(subframe.stderr || subframe.error)" class="errors-container without-error" ng-class="::{'without-error': !subframe.error}"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stderr">Standard Error Stream:</h3> <!----><pre class="stderr" ng-if="::subframe.stderr" ng-bind-html="::subframe.stderr | secureOutput | promptOutput | trustHtml"><div class="outputLine">command 0 4</div></pre><!----> <!----> </div><!----> <div class="frame-hover" ng-style="::{'background-color': subframe.agent.color}"></div> <!----><div ng-if="::subframe.stdout" class="stdout"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.stdout">Standard Output Stream:</h3> <pre ng-bind-html="::subframe.stdout | secureOutput | promptOutput | trustHtml"><div class="outputLine">0 4</div></pre> </div><!----> <!----><div ng-if="::subframe.gameInformation" class="game-info"> <h3 class="title" ng-style="::{color: subframe.agent.color}" translate="ideConsole.gameInfo">Game information:</h3> <pre ng-bind-html="::subframe.gameInformation | formatConsole:agents | trustHtml"><span class="consoleSuccess">Success: Mars Lander landed hard but Opportunity is ok!</span>
X=2500m, Y=100m, HSpeed=0m/s VSpeed=-39m/s
Fuel=283l, Angle=0°, Power=4 (4.0m/s2)
</pre> </div><!----> <div class="frame-number-bloc" ng-style="::{color: getFrameNumberBlockColor(frame)}" ng-click="gotoFrame(frame.idx)"> <span class="frame-number" ng-bind="::frame.idx | zeroPad:(frames.length - 1).toString().length">69</span> <span class="frame-count" ng-bind="::frames.length - 1">69</span> </div> </div><!----><!----> <!----> </div><!----> </div> </div> </div> </div><!----> <!----> <!----> </div> </div> </div>
''';
